In [1]:
'''
TODO:
- rag database, có nối với phương thức nào đó để sau dễ rag
- function thực hiện rag
- function thực hiện predict
- nâng cao hơn, tự quyết định khi nào rag, khi nào không
'''

'\nTODO:\n- rag database, có nối với phương thức nào đó để sau dễ rag\n- function thực hiện rag\n- function thực hiện predict\n- nâng cao hơn, tự quyết định khi nào rag, khi nào không\n'

In [1]:
from openai import OpenAI

def call_llm(prompt, model="gpt-4o-mini"):
    client = OpenAI()
    response = client.chat.completions.create(  
        model='gpt-4o-mini',
        messages=[
            {"role": "user", "content": prompt},
        ]
    )
    return response.choices[0].message.content

print(call_llm("What is the capital of France?"))

The capital of France is Paris.


In [3]:
from langchain.document_loaders import PyMuPDFLoader, UnstructuredWordDocumentLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import ElasticsearchStore
from elasticsearch import Elasticsearch

import os
from glob import glob
'''
Load document và chia thành các chunk để đẩy vào elasticsearch
'''
# def load_documents_from_folder(folder_path):
#     documents = []
#     for file in glob(os.path.join(folder_path, "*")):
#         if file.endswith(".pdf"):
#             loader = PyMuPDFLoader(file)
#         elif file.endswith(".docx"):
#             loader = UnstructuredWordDocumentLoader(file)
#         else:
#             continue
#         docs = loader.load()
#         documents.extend(docs)
#     return documents

# def split_documents(documents):
#     splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
#     return splitter.split_documents(documents)


# folder_path = "data"
# docs = load_documents_from_folder(folder_path)
# split_docs = split_documents(docs)



'\nLoad document và chia thành các chunk để đẩy vào elasticsearch\n'

In [ ]:
'''
đẩy các chunk documents vào elasticsearch
'''

# def create_elasticsearch_store(docs):
#     embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

#     elastic_vector_store = ElasticsearchStore.from_documents(
#         documents=docs,
#         embedding=embeddings,
#         es_cloud_id="agent_elastic:YXNpYS1zb3V0aGVhc3QxLmdjcC5lbGFzdGljLWNsb3VkLmNvbTo0NDMkMTVhN2M0YjQ2MjRhNDI3MjkzYjM5ZWNjMGYzNWRlNTEkYzYyNzdhOWJhMTFlNDBkOGI3NDg4NTFlNmVlYWUzNmU=",
#         index_name="my_index",
#         es_user="elastic",
#         es_password="dfbrEbfnW6RHG0hTFHouCblj",
#     )
#     return elastic_vector_store
# elasticsearch_store = create_elasticsearch_store(split_docs)

In [4]:
''' 
Load elastic store để thực hiện truy vấn
'''
es_connection = Elasticsearch(
    cloud_id="agent_elastic:YXNpYS1zb3V0aGVhc3QxLmdjcC5lbGFzdGljLWNsb3VkLmNvbTo0NDMkMTVhN2M0YjQ2MjRhNDI3MjkzYjM5ZWNjMGYzNWRlNTEkYzYyNzdhOWJhMTFlNDBkOGI3NDg4NTFlNmVlYWUzNmU=",
    basic_auth=("elastic", "dfbrEbfnW6RHG0hTFHouCblj"),
)

elasticsearch_store = ElasticsearchStore(
    embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"),
    index_name="my_index",
    es_connection=es_connection,
)

d:\AI_CODE\Cloud-Agent-API\agent_cloud\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\tridu\AppData\Local\Temp\ipykernel_15892\3712738736.py:9: LangChainPendingDeprecationWarning: The class `ElasticsearchStore` will be deprecated in a future version. Use :class:`~Use class in langchain-elasticsearch package` instead.
  elasticsearch_store = ElasticsearchStore(


In [5]:
def retrieve(query: str, k = 3):
    results = elasticsearch_store.similarity_search(query, k=k)
    return results

In [6]:
### RAG FLOW
def rag(query: str):
    # 1. Retrieve relevant documents
    retrieved_docs = retrieve(query)
    
    # 2. Combine the retrieved documents with the query
    combined_input = f"Query: {query}\nDocuments: {retrieved_docs}"
    
    # 3. Call the LLM with the combined input
    response = call_llm(combined_input)
    
    return response

In [7]:
result = rag("bảo hiểm y tế có những quy định gì quan trọng")

In [8]:
result

'Bảo hiểm y tế tại Việt Nam có một số quy định quan trọng được ghi nhận trong các văn bản pháp luật. Dưới đây là những điểm chính:\n\n1. **Quy định về tổ chức và hoạt động**: Bảo hiểm y tế được tổ chức để đảm bảo quyền lợi chăm sóc sức khỏe cho nhân dân, có sự quản lý và giám sát từ Hội đồng quản lý bảo hiểm xã hội.\n\n2. **Trách nhiệm của Hội đồng quản lý**: Hội đồng quản lý bảo hiểm xã hội, bao gồm cả bảo hiểm y tế, có trách nhiệm chỉ đạo, giám sát và tư vấn chính sách liên quan đến bảo hiểm.\n\n3. **Các quyền lợi được hưởng**: Người tham gia bảo hiểm y tế sẽ được hưởng các quyền lợi chăm sóc sức khỏe theo quy định pháp luật, bao gồm việc được khám chữa bệnh, điều trị và cấp thuốc.\n\n4. **Thời gian làm việc và nghỉ ngơi**: Có quy định liên quan đến thời giờ làm việc và nghỉ ngơi, bảo đảm quyền lợi cho người lao động nói chung, bao gồm cả việc tham gia bảo hiểm y tế.\n\n5. **Đào tạo và nâng cao kỹ năng**: Trong bối cảnh bảo hiểm y tế, có quy định về việc trang bị kiến thức cho người 